In [16]:
import sys
# TO CHANGE
BASEDIR = "../../../.."
sys.path.insert(0, BASEDIR)

In [17]:
from pprint import pprint

In [18]:
from src.db_drivers.vector_driver import EmbedderModelConfig
from src.kg_model import KnowledgeGraphModel, KnowledgeGraphModelConfig
from src.agents import AgentDriverConfig
from src.utils import TripletCreator, NodeCreator, RelationCreator
from src.agents.configs import DEFAULT_AGENT_CONFIGS
from src.kg_model.utils import KGEmbeddersMapping, AgentsMapping

## Пример работы с моделью графа знаний

1. Инициализация класса, выполняющего функции embedder-модели

In [4]:
EMBEDDER_MODEL_PATH = '../../../../models/intfloat/multilingual-e5-small' # PATH TO APPROPRIATE EMBEDDER-MODEL
embedder_config = EmbedderModelConfig(model_name_or_path=EMBEDDER_MODEL_PATH)
pprint(embedder_config)

EmbedderModelConfig(model_name_or_path='../../../../models/intfloat/multilingual-e5-small',
                    prompts={'passage': 'passage: ', 'query': 'query: '},
                    device='cuda',
                    normalize_embeddings=True)


2. Инициализация Connector-класса к LLM-моделе

In [5]:
adriver_config = AgentDriverConfig(name='ollama', agent_config=DEFAULT_AGENT_CONFIGS['ollama'])
pprint(adriver_config, width=200)

AgentDriverConfig(name='ollama',
                  agent_config=AgentConnectorConfig(gen_strategy={'num_predict': 2048, 'seed': 42, 'temperature': 0.0, 'top_k': 1},
                                                    credentials={'host': 'localhost', 'model': 'llama3.1:8b', 'port': 11437},
                                                    ext_params={'keep_alive': -1, 'timeout': 560}))


3. Спецификация embedder- и llm-моделей, которые будут использоваться в компонентах модели графа знаний

In [6]:
EMBEDDERS_CONFIG = {
    'm-e5-small': embedder_config
}

EMBEDDERS_MAP = KGEmbeddersMapping(
    embeddings_model={'nodes_dense': 'm-e5-small', 'triplets_dense': 'm-e5-small'}
)

AGENTS_CONFIG = {
    'llama3.1:8b': adriver_config 
}

AGENTS_MAP = AgentsMapping(
    qa_pipeline='llama3.1:8b',
    mem_pipeline='llama3.1:8b',
    kg_nodestree_model='llama3.1:8b'
)

4. Инициализация модели графа знаний

In [7]:
kg_config = KnowledgeGraphModelConfig(
    agents_configs=AGENTS_CONFIG,
    embedders_configs=EMBEDDERS_CONFIG,
    agents_map=AGENTS_MAP,
    embedders_map=EMBEDDERS_MAP,
    nodestree_config=None # Модель дерева вершин строиться не будет
) 

In [19]:
kg_model = KnowledgeGraphModel(kg_config)

No sentence-transformers model found with name ../../../../models/intfloat/multilingual-e5-small. Creating a new one with mean pooling.


In [21]:
pprint(kg_model.count_items(detailed=True))

{'embeddings_info': {'nodes': {'episodic': {'nodes_dense': 0,
                                            'nodes_sparse_bm25': 0},
                               'hyper': {'nodes_dense': 0,
                                         'nodes_sparse_bm25': 0},
                               'object': {'nodes_dense': 4,
                                          'nodes_sparse_bm25': 4}},
                     'triplets': {'triplets_dense': 2,
                                  'triplets_sparse_bm25': 2}},
 'graph_info': {'nodes': {'episodic': 0, 'hyper': 0, 'object': 4},
                'triplets': {'episodic': 0, 'hyper': 0, 'simple': 2}},
 'nodestree_info': None}


5. Добавление триплетов в модель графа знаний

In [11]:
TRIPLET_EXAMPLE1 = TripletCreator.create(
    start_node=NodeCreator.create(n_type='object', name='саша'), 
    relation=RelationCreator.create(r_type='simple', name='шла'), 
    end_node=NodeCreator.create(n_type='object', name='улица'))
TRIPLET_EXAMPLE2 = TripletCreator.create(
    start_node=NodeCreator.create(n_type='object', name='маша'), 
    relation=RelationCreator.create(r_type='simple', name='шла'), 
    end_node=NodeCreator.create(n_type='object', name='шоссе'))
TRIPLET_EXAMPLE3 = TripletCreator.create(
    start_node=NodeCreator.create(n_type='object', name='теплоход'), 
    relation=RelationCreator.create(r_type='simple', name='плыл'), 
    end_node=NodeCreator.create(n_type='object', name='канал'))
TRIPLET_EXAMPLE4 = TripletCreator.create(
    start_node=NodeCreator.create(n_type='object', name='моторная лодка'), 
    relation=RelationCreator.create(r_type='simple', name='плыл'), 
    end_node=NodeCreator.create(n_type='object', name='река'))

In [13]:
add_info = kg_model.add_knowledge([TRIPLET_EXAMPLE1, TRIPLET_EXAMPLE2, TRIPLET_EXAMPLE3, TRIPLET_EXAMPLE4], check_consistency=True, status_bar=True)
print("Информация о добавленных триплетах: ")
pprint(add_info)
print("Количество элементов в моделе графа знаний:")
pprint(kg_model.count_items())

100%|██████████| 1/1 [00:02<00:00,  2.96s/it]

Информация о добавленных триплетах: 
{'embeddings_info': {'nodes': {'2401d7e9c775689eceb9108c20060b0e',
                               '64ec88e2cb968f3b8c55f5e7316f0418',
                               '72adc4fc2f53b1344fb5b56d6ed81b1f',
                               '956c2d5e231f655cee06efa6a32969e9',
                               'b8f8c47ba32fcbf247f40e993965ad01',
                               'd51ca416731c328c6bb159cc30803659',
                               'eebef31909a5d48ab3d1b7f3cd507df4',
                               'f7670dc5605796e6f98a48a3d5d784b4'},
                     'triplets': {'8c3274995b6a321367f6986fe3f1bace',
                                  'c2c994676e30850fdf95f0f2a647ba30',
                                  'cd10ac9474cf7879972bda116a5e05a8',
                                  'd780616b74075f112a4b5b0fde3d807b'}},
 'graph_info': {'nodes': {'2401d7e9c775689eceb9108c20060b0e',
                          '64ec88e2cb968f3b8c55f5e7316f0418',
                    

6. Удаление триплетов из модели графа знаний

In [14]:
remove_info = kg_model.remove_knowledge([TRIPLET_EXAMPLE2, TRIPLET_EXAMPLE4])
print("Информация об удалённых триплетах: ")
pprint(remove_info)
print("Количество элементов в моделе графа знаний:")
pprint(kg_model.count_items())

Информация об удалённых триплетах: 
{'embeddings_info': {0: {'e_node': True, 's_node': True, 'triplet': True},
                     1: {'e_node': True, 's_node': True, 'triplet': True}},
 'graph_info': {0: {'e_node': True, 's_node': True},
                1: {'e_node': True, 's_node': True}},
 'tree_info': None}
Количество элементов в моделе графа знаний:
{'embeddings_info': {'nodes': 4, 'triplets': 2},
 'graph_info': {'nodes': 4, 'triplets': 2},
 'nodestree_info': None}


7. Полное удаление содержимого модели графа знаний

In [12]:
kg_model.clear()
print("Количество элементов в моделе графа знаний:")
pprint(kg_model.count_items())
print("Консистентность модели графа знаний:", kg_model.check_consistency())

Количество элементов в моделе графа знаний:
{'embeddings_info': {'nodes': 0, 'triplets': 0},
 'graph_info': {'nodes': 0, 'triplets': 0},
 'nodestree_info': None}
Консистентность модели графа знаний: True


In [15]:
del kg_model

closing inmemory bm25 connection...
inmemory bm25-store saved in: ./personalai_tmp/memory_parts/embeddings_model/vectorized_nodes/inmemory_bm25/sparsebm25_nodes_object.json
closing inmemory bm25 connection...
closing inmemory bm25 connection...
inmemory bm25-store saved in: ./personalai_tmp/memory_parts/embeddings_model/vectorized_nodes/inmemory_bm25/sparsebm25_nodes_hyper.json
closing inmemory bm25 connection...
closing inmemory bm25 connection...
inmemory bm25-store saved in: ./personalai_tmp/memory_parts/embeddings_model/vectorized_nodes/inmemory_bm25/sparsebm25_nodes_episodic.json
closing inmemory bm25 connection...
closing inmemory bm25 connection...
inmemory bm25-store saved in: ./personalai_tmp/memory_parts/embeddings_model/vectorized_triplets/inmemory_bm25/sparsebm25_triplets.json
closing inmemory bm25 connection...
